<a href="https://colab.research.google.com/github/nrana00/Assignment-for-Classification/blob/main/Classification_using_Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Check the environment
!java --version
!phython --version

openjdk 11.0.19 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
/bin/bash: phython: command not found


In [5]:
# Download Apache Spark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=cd42b708491e8f49779309725b575b56948ca880c6fd9c85ad62d8e469850de5
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [6]:
# Initiate the Spark Session
from pyspark.sql import SparkSession

# Create Spark Session/Context
spark = SparkSession.builder \
  .master("local") \
  .appName("Hello Pyspark") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

print(spark)

The Adult dataset is publicly available at the UCI Machine Learning Repository. This data derives from census data and consists of information about 48842 individuals and their annual income. You can use this information to predict if an individual earns <=50K or >50k a year. The dataset consists of both numeric and categorical variables.

Attribute Information:

age: continuous
workclass: Private,Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked
fnlwgt: continuous
education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc...
education-num: continuous
marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent...
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners...
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried
race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black
sex: Female, Male
capital-gain: continuous
capital-loss: continuous
hours-per-week: continuous
native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany...
Target/Label: - <=50K, >50K

In [12]:
from pyspark.sql.types import DoubleType, StringType, StructField, StructType
 
schema = StructType([
  StructField("age", DoubleType(), False),
  StructField("workclass", StringType(), False),
  StructField("fnlwgt", DoubleType(), False),
  StructField("education", StringType(), False),
  StructField("education_num", DoubleType(), False),
  StructField("marital_status", StringType(), False),
  StructField("occupation", StringType(), False),
  StructField("relationship", StringType(), False),
  StructField("race", StringType(), False),
  StructField("sex", StringType(), False),
  StructField("capital_gain", DoubleType(), False),
  StructField("capital_loss", DoubleType(), False),
  StructField("hours_per_week", DoubleType(), False),
  StructField("native_country", StringType(), False),
  StructField("income", StringType(), False)
])
 


In [15]:
dataset = spark.read.format("csv").schema(schema).load("adult.data")
cols = dataset.columns

In [16]:
dataset = spark.read.format("csv").schema(schema).load("adult.data")
cols = dataset.columns

In [17]:
dataset.show()

+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
| age|        workclass|  fnlwgt|    education|education_num|      marital_status|        occupation|  relationship|               race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
|39.0|        State-gov| 77516.0|    Bachelors|         13.0|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|
|50.0| Self-emp-not-inc| 83311.0|    Bachelors|         13.0|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|         0.0|         0.

In [18]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
 
from distutils.version import LooseVersion
 
categoricalColumns = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country"]
stages = [] # stages in Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    if LooseVersion(pyspark.__version__) < LooseVersion("3.0"):
        from pyspark.ml.feature import OneHotEncoderEstimator
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    else:
        from pyspark.ml.feature import OneHotEncoder
        encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]

In [19]:
stages = [] # stages in Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

In [20]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol="income", outputCol="label")
stages += [label_stringIdx]

In [21]:
stages

[StringIndexer_ff2ea2c5c7aa,
 OneHotEncoder_75f8a3509411,
 StringIndexer_e3c56f765d1b,
 OneHotEncoder_f26fc0055ad6,
 StringIndexer_62634d610bb1,
 OneHotEncoder_19bc88fdef8f,
 StringIndexer_c38d6fc8c7b4,
 OneHotEncoder_088bbd10ef0a,
 StringIndexer_7bb76a8a6c0b,
 OneHotEncoder_9c53079aa301,
 StringIndexer_936136229d45,
 OneHotEncoder_34e16a5a3fa0,
 StringIndexer_8c75ebe16a3f,
 OneHotEncoder_38c7b749df95,
 StringIndexer_dfcc5c8f75d5,
 OneHotEncoder_7f3e6b5384c6,
 StringIndexer_b0d5076a5db3]

In [22]:
# Transform all features into a vector using VectorAssembler
numericCols = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [23]:
from pyspark.ml.classification import LogisticRegression
  
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(dataset)
preppedDataDF = pipelineModel.transform(dataset)

In [24]:
preppedDataDF.show()

+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+--------------+-----------------+--------------+-----------------+-------------------+----------------------+---------------+------------------+-----------------+--------------------+---------+-------------+--------+-------------+-------------------+----------------------+-----+--------------------+
| age|        workclass|  fnlwgt|    education|education_num|      marital_status|        occupation|  relationship|               race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|workclassIndex|workclassclassVec|educationIndex|educationclassVec|marital_statusIndex|marital_statusclassVec|occupationIndex|occupationclassVec|relationshipIndex|relationshipclassVec|raceIndex| raceclassVec|sexIndex|  sexclassVec|native_countryIndex|native_countryclassVec|label|      

In [25]:
# Fit model to prepped data
lrModel = LogisticRegression().fit(preppedDataDF)
 
# ROC for training data
display(lrModel, preppedDataDF, "ROC")

LogisticRegressionModel: uid=LogisticRegression_eaf2a86550d8, numClasses=2, numFeatures=100

DataFrame[age: double, workclass: string, fnlwgt: double, education: string, education_num: double, marital_status: string, occupation: string, relationship: string, race: string, sex: string, capital_gain: double, capital_loss: double, hours_per_week: double, native_country: string, income: string, workclassIndex: double, workclassclassVec: vector, educationIndex: double, educationclassVec: vector, marital_statusIndex: double, marital_statusclassVec: vector, occupationIndex: double, occupationclassVec: vector, relationshipIndex: double, relationshipclassVec: vector, raceIndex: double, raceclassVec: vector, sexIndex: double, sexclassVec: vector, native_countryIndex: double, native_countryclassVec: vector, label: double, features: vector]

'ROC'

In [26]:
display(lrModel, preppedDataDF)

LogisticRegressionModel: uid=LogisticRegression_eaf2a86550d8, numClasses=2, numFeatures=100

DataFrame[age: double, workclass: string, fnlwgt: double, education: string, education_num: double, marital_status: string, occupation: string, relationship: string, race: string, sex: string, capital_gain: double, capital_loss: double, hours_per_week: double, native_country: string, income: string, workclassIndex: double, workclassclassVec: vector, educationIndex: double, educationclassVec: vector, marital_statusIndex: double, marital_statusclassVec: vector, occupationIndex: double, occupationclassVec: vector, relationshipIndex: double, relationshipclassVec: vector, raceIndex: double, raceclassVec: vector, sexIndex: double, sexclassVec: vector, native_countryIndex: double, native_countryclassVec: vector, label: double, features: vector]

In [27]:
# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = preppedDataDF.select(selectedcols)
dataset.show()

+-----+--------------------+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
|label|            features| age|        workclass|  fnlwgt|    education|education_num|      marital_status|        occupation|  relationship|               race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+-----+--------------------+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
|  0.0|(100,[4,10,24,32,...|39.0|        State-gov| 77516.0|    Bachelors|         13.0|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|
|  0.0|(100,[1,10,23,31,...|50.0| Self-emp-not-inc| 83311.0|

In [28]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())

22832
9729


In [29]:
from pyspark.ml.classification import LogisticRegression
 
# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)
 
# Train model with Training Data
lrModel = lr.fit(trainingData)

In [30]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)

In [31]:
selected = predictions.select("label", "prediction", "income", "probability", "age", "occupation")
selected.show()

+-----+----------+------+--------------------+----+---------------+
|label|prediction|income|         probability| age|     occupation|
+-----+----------+------+--------------------+----+---------------+
|  0.0|       1.0| <=50K|[0.16646708724833...|36.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.70012031106347...|32.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.53453682932965...|33.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.67559303222185...|39.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.62237992652147...|39.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.61047384917734...|50.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.60270631574160...|51.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.60470503143805...|60.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.74230866284291...|34.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.98954486365582...|20.0| Prof-specialty|
|  0.0|       1.0| <=50K|[0.35443820573118...|35.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.52012311192879...|42

In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
 
# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.9010633018702178

In [33]:
evaluator.getMetricName()

'areaUnderROC'

In [34]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The

In [35]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
 
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [36]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
 
# Run cross validations
cvModel = cv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

In [37]:
predictions = cvModel.transform(testData)

In [38]:
evaluator.evaluate(predictions)

0.8997203469558716

In [39]:
print('Model Intercept: ', cvModel.bestModel.intercept)

Model Intercept:  -6.336365999298099


In [40]:
display((cvModel).bestModel.explainParams())

"aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)\nelasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0, current: 0.0)\nfamily: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)\nfeaturesCol: features column name. (default: features, current: features)\nfitIntercept: whether to fit an intercept term. (default: True)\nlabelCol: label column name. (default: label, current: label)\nlowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)\nlowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constra

In [41]:
weights = cvModel.bestModel.coefficients
weights = [(float(w),) for w in weights]  # convert numpy type to float, and to tuple
weightsDF = spark.createDataFrame(weights, ["Feature Weight"])
display(weightsDF)

DataFrame[Feature Weight: double]

In [42]:
# View best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "income" , "probability", "age", "occupation")
selected.show()

+-----+----------+------+--------------------+----+---------------+
|label|prediction|income|         probability| age|     occupation|
+-----+----------+------+--------------------+----+---------------+
|  0.0|       1.0| <=50K|[0.21060910009532...|36.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.68097448822474...|32.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.54341491059548...|33.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.65860417846993...|39.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.61463593853443...|39.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.60241279947003...|50.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.59586897256623...|51.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.59681289807548...|60.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.72170646267586...|34.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.96796930272851...|20.0| Prof-specialty|
|  0.0|       1.0| <=50K|[0.47756430733627...|35.0| Prof-specialty|
|  0.0|       0.0| <=50K|[0.52627860330667...|42

Decision tree

In [43]:
from pyspark.ml.classification import DecisionTreeClassifier
 
# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)
 
# Train model with Training Data
dtModel = dt.fit(trainingData)

In [44]:
print("numNodes = ", dtModel.numNodes)
print("depth = ", dtModel.depth)

numNodes =  11
depth =  3


In [45]:
display(dtModel)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_5d237169508e, depth=3, numNodes=11, numClasses=2, numFeatures=100

In [46]:
# Make predictions on test data using the Transformer.transform() method.
predictions = dtModel.transform(testData)

In [47]:
predictions.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- age: double (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [58]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
selected.show()


+-----+----------+--------------------+----+---------------+
|label|prediction|         probability| age|     occupation|
+-----+----------+--------------------+----+---------------+
|  0.0|       0.0|[0.83333333333333...|36.0| Prof-specialty|
|  0.0|       0.0|[0.84040747028862...|32.0| Prof-specialty|
|  0.0|       0.0|[0.73248407643312...|33.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|39.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|39.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|50.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|51.0| Prof-specialty|
|  0.0|       0.0|[0.89389067524115...|60.0| Prof-specialty|
|  0.0|       0.0|[0.84040747028862...|34.0| Prof-specialty|
|  0.0|       0.0|[0.91361256544502...|20.0| Prof-specialty|
|  0.0|       0.0|[0.84040747028862...|35.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|42.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|43.0| Prof-specialty|
|  0.0|       0.0|[0.739

In [49]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.7389215983587255

In [50]:
dt.getImpurity()

'gini'

In [51]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1, 2, 6, 10])
             .addGrid(dt.maxBins, [20, 40, 80])
             .build())

In [52]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
 
# Run cross validations
cvModel = cv.fit(trainingData)
# Takes ~5 minutes

In [53]:
print("numNodes = ", cvModel.bestModel.numNodes)
print("depth = ", cvModel.bestModel.depth)

numNodes =  389
depth =  10


In [54]:
# Use test set to measure the accuracy of the model on new data
predictions = cvModel.transform(testData)

In [55]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

0.7847131905029388

In [57]:
# View Best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
selected.show()


+-----+----------+--------------------+----+---------------+
|label|prediction|         probability| age|     occupation|
+-----+----------+--------------------+----+---------------+
|  0.0|       0.0|[0.83333333333333...|36.0| Prof-specialty|
|  0.0|       0.0|[0.84040747028862...|32.0| Prof-specialty|
|  0.0|       0.0|[0.73248407643312...|33.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|39.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|39.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|50.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|51.0| Prof-specialty|
|  0.0|       0.0|[0.89389067524115...|60.0| Prof-specialty|
|  0.0|       0.0|[0.84040747028862...|34.0| Prof-specialty|
|  0.0|       0.0|[0.91361256544502...|20.0| Prof-specialty|
|  0.0|       0.0|[0.84040747028862...|35.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|42.0| Prof-specialty|
|  0.0|       0.0|[0.65073746312684...|43.0| Prof-specialty|
|  0.0|       0.0|[0.739